In [1]:
import tecplot as tp
import numpy as np
import pandas as pd
from tecplot.constant import *
from tecplot_error import *
from tecplot_axes import *
#导入初始化
tp.session.connect()
#求解速度场
def velocity_field():
    '''
    数据转换函数，使用方式参考：
    二氧化碳浓度场：
    tp.data.operate.execute_equation('{CO2 (ppm)} = {Mass fraction of co2}*1E6')
    '''
    tp.data.operate.execute_equation('{Velocity (m/s)} = SQRT({X Velocity}**2 + {Y Velocity}**2 + {Z Velocity}**2)')

#创建剖切面云图
def creat_slice(frame = tp.active_frame(),variable = 'Static Pressure',
                translucency = True,shade = False,slices = True):
    plot = frame.plot()
    plot.use_translucency = translucency
    plot.contour(0).variable = dataset.variable(variable)
    plot.show_shade = shade
    plot.show_slices = slices
    plot.slice(0).orientation = SliceSurface.XPlanes
    plot.slice(1).orientation = SliceSurface.YPlanes
    plot.slice(2).orientation = SliceSurface.ZPlanes
    slices = plot.slices(0,1,2)
    slices.show = True
    slices.edge.show = True
    slices.edge.line_thickness = 0.25
    slices.effects.use_translucency = True
    return slices

#重置剖切面位置
def reset_slices(plot = tp.active_frame().plot(),variable = 'Static Pressure'):
    axes = plot.axes
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    slices = creat_slice(variable=variable)
    temp = ()
    j = 0
    for each in axes:
        temp = temp + ((each.max + each.min)/2,)
        init_origin = [0,0,0]
        init_origin[j] = temp[j]
        slices[j].origin = tuple(init_origin)
        j += 1

#云图图例格式化
def legend_init(frame = tp.active_frame(),fonts = 'Time New Roman'):
    legend = frame.plot().contour(0).legend
    legend.show = True
    legend.vertical = False
    legend.label_location = ContLegendLabelLocation.ContourLevels
    legend.position = (100,100)
    legend.box.box_type = tp.constant.TextBox.None_
    legend.number_font.typeface = fonts
    legend.number_font.size = 2.5
    legend.header.font.typeface = fonts
    legend.header.font.size = 2.5

#方向标格式化
def axes_init(frame = tp.active_frame()):
    plot = frame.plot()
    plot.axes.orientation_axis.show = True
    plot.axes.orientation_axis.position = 7,7
    plot.axes.orientation_axis.size = 7

#单个方向剖面图重置
def reset_one_slice(direction = 'x'):
    slices = tp.active_frame().plot().slices(0,1,2)
    direction_dict = {'x':0,'y':1,'z':2}
    vec = direction_dict[direction]
    axes = tp.active_frame().plot().axes
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    axes_range = (l_axes[vec].max + l_axes[vec].min)/2
    init_origin = [0,0,0]
    init_origin[vec] = axes_range
    slices[vec].origin = tuple(init_origin)

#各方向自定义云图
def modify_slice(direction = 'x',num = 0):
    direciton_dict = {'x':0,'y':1,'z':2}
    vec = direciton_dict[direction]
    frame = tp.active_frame()
    plot = frame.plot()
    axes = plot.axes
    slices = plot.slices(0,1,2) #0,1,2分别对应x,y,z方向
    slices[vec].show = True
    init_slices = [0,0,0]
    init_slices[vec] = num
    slices[vec].origin = tuple(init_slices)
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    if num > l_axes[0].max or num < l_axes[0].min:
        reset_one_slice('x')
        print('Range: %.2f to %.2f' %(l_axes[0].min,l_axes[0].max))
        print('Now number: %.2f  Out of range of coordinates !' %(num))

#去除数据点集
def clear_extract(name = 'line',output = False):
    while True:
        try:
            dataset.delete_zones(dataset.zone(name))
        except:
            if output:
                print('Clear complete')
                break
            break

#返回朝向剖切面
def direciton_vec(direction = 'x'):
    direciton_dict = {'x':0,'y':1,'z':2}
    if type(direction) == str:
        vec = direciton_dict[direction]
        slices = tp.active_frame().plot().slices(0,1,2)
        return slices[vec]
    elif type(direction) == list:
        res = []
        for each in direction:
            res.append(direciton_vec(each))
            return res

# 线数据提取
def line_data_extract(position = [1.67,-3.8,1.67],direction = 'x',stop = 5,step = 1,variable = 'Static Pressure',clear = True,extarct_name = 'line_extract'):
    plot = frame.plot()
    plot.show_scatter = True
    axes = plot.axes
    if clear:
        clear_extract('line_extract')
        clear_extract('Extracted Points')
    direciton_dict = {'x':0,'y':1,'z':2}
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    position_res = []
    position_origin = position # 该处Y、Z坐标与Probe中的Y、Z坐标相反
    position_matrix = np.arange(position_origin[direciton_dict[direction]],stop,step)
    out_of_range_check(direction,position_matrix[-1])
    for i in position_matrix:
        position_origin[direciton_dict[direction]] = i
        position_res.append(position_origin.copy()) # 需要使用copy()函数，否则position_origin会被修改
    line = tp.data.extract.extract_line(position_res)
    variables = line.values(variable)
    if len(variables) != len(position_res):
        raise Exception('The number of data points is not equal to the number of variables ! %s Out of range' %(position_matrix[-1]))
    pd_charts = pd.DataFrame(columns=['Coordinate Point','Static Pressure'])
    for i in range(len(variables)):
        pd_charts.loc[i] = [position_res[i],variables[i]]
    line.name = extarct_name
    return pd_charts

#矩阵数据提取（仅支持轴向）
def matrix_data_extract(origin=[1.67, -3.8, 1.67], target={'x': 4, 'y': 4}, step={'x': 1, 'y': 1}, variable='Static Pressure', clear=False, extarct_name='matrix_extract'):
    direcition_dict = {'x': 0, 'y': 1, 'z': 2}
    direction = list(target.keys())
    target = list(target.values())
    step = list(step.values())
    position_matrix = []
    position_step = np.arange(origin[direcition_dict[direction[1]]], target[1], step[1])
    for i in position_step:
        origin[direcition_dict[direction[1]]] = i
        line_data_extract(position=origin.copy(), direction=direction[0], stop=target[0],
                          step=step[0], variable=variable, clear=clear, extarct_name=extarct_name)

Connecting to Tecplot 360 TecUtil Server on:
    tcp://localhost:7600
Connection established.


In [2]:
# 初始化步骤
tp.session.connect()
frame = tp.active_frame()
plot = frame.plot()
dataset = frame.dataset
fmaps = frame.plot().fieldmaps([dataset.zone(i) for i in range(frame.plot().num_fieldmaps)])
fmaps.scatter.show = False #关闭原始点数据显示

# axes_transform()
velocity_field()
creat_slice(variable='Velocity (m/s)')
legend_init()
axes_init()
axes_fix()
reset_slices(variable='Velocity (m/s)')
# 数据提取
'''
数据提取示例

线数据：
line_data_extract(position = [1.67,-3.8,1.67],direction = 'x',stop = 4,step = 1,variable = 'Static Pressure')

矩阵数据：

'''

Connecting to Tecplot 360 TecUtil Server on:
    tcp://localhost:7600
Connection established.


"\n数据提取示例\n\n线数据：\nline_data_extract(position = [1.67,-3.8,1.67],direction = 'x',stop = 4,step = 1,variable = 'Static Pressure')\n\n矩阵数据：\n\n"

In [4]:
import inspect
import importlib # 动态导入模块(避免jupyter在更新模块时需要重启kernel)
from tecplot_plot import *
importlib.reload(inspect.getmodule(creat_streamtraces))
creat_streamtraces(replace=True,zone_name='inlet_mouth')

In [11]:
modify_slice('z',num=1.447)
plot = tp.active_frame().plot()
plot.view.rotate_to_angles(0,0,0)
plot.axes.axis_mode = AxisMode.Independent
plot.axes.x_axis.show = True
plot.axes.y_axis.show = True
plot.axes.x_axis.min = -1
plot.axes.x_axis.max = 1
plot.axes.y_axis.min = -1
plot.axes.y_axis.max = 1
plot.axes.x_axis.scale_factor = 1
plot.axes.y_axis.scale_factor = 1
plot.axes.z_axis.scale_factor = 1
plot.slices(0,1,2).show = False
plot.view.width = 3

In [5]:
frame = tp.active_frame()
frame.position=(9,0)
frame.width = 9
frame.height = 8

In [10]:
axes = frame.plot().axes
axes.x_axis.fit_range()
axes.y_axis.fit_range()
axes.z_axis.fit_range()